In [1]:
import pandas as pd
import numpy as np

# Import the first CSV file
cleaned_df = pd.read_csv('ufc_cleaned.csv')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
cleaned_df.head()

,p1_fighter,p2_fighter,p1_KD,p2_KD,p1_SIG_STR_PCT,p2_SIG_STR_PCT,p1_TD_PCT,p2_TD_PCT,p1_SUB_ATT,p2_SUB_ATT,p1_REV,p2_REV,p1_CTRL,p2_CTRL,p1_R1_KD,p2_R1_KD,p1_R1_SIG_STR_PCT,p2_R1_SIG_STR_PCT,p1_R1_TD_PCT,p2_R1_TD_PCT,p1_R1_SUB_ATT,p2_R1_SUB_ATT,p1_R1_REV,p2_R1_REV,p1_R1_CTRL,p2_R1_CTRL,p1_SIG_STR_PCT_DETAILED,p2_SIG_STR_PCT_DETAILED,p1_R1_SIG_STR_PCT_DETAILED,p2_R1_SIG_STR_PCT_DETAILED,method,referee,winner,event_date,p1_SIG_STR_LANDED,p1_SIG_STR_ATTEMPTED,p2_SIG_STR_LANDED,p2_SIG_STR_ATTEMPTED,p1_TOTAL_STR_LANDED,p1_TOTAL_STR_ATTEMPTED,p2_TOTAL_STR_LANDED,p2_TOTAL_STR_ATTEMPTED,p1_TD_LANDED,p1_TD_ATTEMPTED,p2_TD_LANDED,p2_TD_ATTEMPTED,p1_R1_SIG_STR_LANDED,p1_R1_SIG_STR_ATTEMPTED,p2_R1_SIG_STR_LANDED,p2_R1_SIG_STR_ATTEMPTED,p1_R1_TOTAL_STR_LANDED,p1_R1_TOTAL_STR_ATTEMPTED,p2_R1_TOTAL_STR_LANDED,p2_R1_TOTAL_STR_ATTEMPTED,p1_R1_TD_LANDED,p1_R1_TD_ATTEMPTED,p2_R1_TD_LANDED,p2_R1_TD_ATTEMPTED,p1_HEAD_LANDED,p1_HEAD_ATTEMPTED,p2_HEAD_LANDED,p2_HEAD_ATTEMPTED,p1_BODY_LANDED,p1_BODY_ATTEMPTED,p2_BODY_LANDED,p2_BODY_ATTEMPTED,p1_LEG_LANDED,p1_LEG_ATTEMPTED,p2_LEG_LANDED,p2_LEG_ATTEMPTED,p1_DISTANCE_LANDED,p1_DISTANCE_ATTEMPTED,p2_DISTANCE_LANDED,p2_DISTANCE_ATTEMPTED,p1_CLINCH_LANDED,p1_CLINCH_ATTEMPTED,p2_CLINCH_LANDED,p2_CLINCH_ATTEMPTED,p1_GROUND_LANDED,p1_GROUND_ATTEMPTED,p2_GROUND_LANDED,p2_GROUND_ATTEMPTED,p1_R1_HEAD_LANDED,p1_R1_HEAD_ATTEMPTED,p2_R1_HEAD_LANDED,p2_R1_HEAD_ATTEMPTED,p1_R1_BODY_LANDED,p1_R1_BODY_ATTEMPTED,p2_R1_BODY_LANDED,p2_R1_BODY_ATTEMPTED,p1_R1_LEG_LANDED,p1_R1_LEG_ATTEMPTED,p2_R1_LEG_LANDED,p2_R1_LEG_ATTEMPTED,p1_R1_DISTANCE_LANDED,p1_R1_DISTANCE_ATTEMPTED,p2_R1_DISTANCE_LANDED,p2_R1_DISTANCE_ATTEMPTED,p1_R1_CLINCH_LANDED,p1_R1_CLINCH_ATTEMPTED,p2_R1_CLINCH_LANDED,p2_R1_CLINCH_ATTEMPTED,p1_R1_GROUND_LANDED,p1_R1_GROUND_ATTEMPTED,p2_R1_GROUND_LANDED,p2_R1_GROUND_ATTEMPTED,p1_height,p1_weight,p1_reach,p1_stance,p1_dob,p1_SLpM,p1_Str. Acc.,p1_SApM,p1_Str. Def,p1_TD Avg.,p1_TD Acc.,p1_TD Def.,p1_Sub. Avg.,p2_height,p2_weight,p2_reach,p2_stance,p2_dob,p2_SLpM,p2_Str. Acc.,p2_SApM,p2_Str. Def,p2_TD Avg.,p2_TD Acc.,p2_TD Def.,p2_Sub. Avg.
0,Mateusz Gamrot,Ludovit Klein,0,0,0.59,0.32,0.54,NaN,0,0,0,0,525,23,0,0,0.52,0.14,0.57,NaN,0,0,0,0,186,0,0.59,0.32,0.52,0.14,Decision - Unanimous,Herb Dean,1,2025-05-31,65,110,24,74,142,224,85,145,6,11,0,0,10,19,2,14,27,43,6,19,4,7,0,0,53,98,10,49,7,7,13,23,5,5,1,2,38,73,23,73,3,4,1,1,24,33,0,0,3,12,2,11,4,4,0,3,3,3,0,0,4,13,2,14,2,2,0,0,4,4,0,0,70.0,155.0,70.0,Southpaw,1990-12-11,3.23,0.50,3.22,0.58,5.25,0.35,0.90,0.1,67.0,155.0,72.0,Southpaw,1995-02-22,3.81,0.54,3.22,0.54,1.70,0.51,0.91,0.0
1,Billy Ray Goff,Ramiz Brahimaj,0,0,0.33,1.00,0.00,0.66,0,1,0,0,59,102,0,0,0.33,1.00,0.00,0.66,0,1,0,0,59,102,0.33,1.00,0.33,1.00,Submission,Chris Tognoni,0,2025-05-31,1,3,1,1,2,4,9,10,0,1,2,3,1,3,1,1,2,4,9,10,0,1,2,3,0,1,0,0,1,2,1,1,0,0,0,0,0,2,0,0,1,1,1,1,0,0,0,0,0,1,0,0,1,2,1,1,0,0,0,0,0,2,0,0,1,1,1,1,0,0,0,0,70.0,170.0,72.0,Switch,1998-06-18,8.56,0.46,5.71,0.59,2.01,0.17,1.00,0.0,70.0,170.0,72.0,Orthodox,1992-11-17,2.26,0.44,4.04,0.43,1.51,0.35,0.44,1.5
2,Dustin Jacoby,Bruno Lopes,1,0,0.58,0.33,NaN,NaN,0,0,0,0,30,0,1,0,0.58,0.33,NaN,NaN,0,0,0,0,30,0,0.58,0.33,0.58,0.33,KO/TKO,Mike Beltran,1,2025-05-31,17,29,3,9,17,30,5,12,0,0,0,0,17,29,3,9,17,30,5,12,0,0,0,0,16,28,1,4,0,0,1,2,1,1,1,3,11,21,2,7,6,8,1,2,0,0,0,0,16,28,1,4,0,0,1,2,1,1,1,3,11,21,2,7,6,8,1,2,0,0,0,0,75.0,205.0,76.0,Orthodox,1988-04-04,5.33,0.47,4.05,0.57,0.33,0.25,0.62,0.0,74.0,205.0,74.0,Orthodox,1993-04-24,2.89,0.43,3.00,0.60,2.68,0.33,1.00,0.0
3,Ketlen Vieira,Macy Chiasson,0,0,0.43,0.51,0.00,0.00,0,0,0,0,606,33,0,0,0.37,0.54,NaN,NaN,0,0,0,0,66,8,0.43,0.51,0.37,0.54,Decision - Unanimous,Mark Smith,1,2025-05-31,28,64,22,43,106,174,52,77,0,1,0,4,18,48,17,31,29,64,18,32,0,0,0,0,25,60,8,25,3,3,6,9,0,1,8,9,15,47,20,39,5,6,2,4,8,11,0,0,15,44,5,16,3,3,5,7,0,1,7,8,13,43,17,29,5,5,0,2,0,0,0,0,68.0,135.0,68.0,Orthodox,1991-08-26,2.92,0.41,3.90,0.51,1.39,0.44,0.88,0.6,71.0,135.0,72.0,Orthodox,1991-07-27,3.77,0.46,2.79,0.44,2.37,0.38,0

In [2]:
#calculate fighters age relative to event
cleaned_df['event_date'] = pd.to_datetime(cleaned_df['event_date'])
cleaned_df['p1_dob'] = pd.to_datetime(cleaned_df['p1_dob'])
cleaned_df['p2_dob'] = pd.to_datetime(cleaned_df['p2_dob'])

cleaned_df['p1_age_at_event'] = (cleaned_df['event_date'] - cleaned_df['p1_dob']).dt.days / 365.25
cleaned_df['p2_age_at_event'] = (cleaned_df['event_date'] - cleaned_df['p2_dob']).dt.days / 365.25

cleaned_df = cleaned_df.drop(columns=['p1_dob', 'p2_dob'])

#compute physical differences
cleaned_df['height_diff'] = cleaned_df['p1_height'] - cleaned_df['p2_height']
cleaned_df['reach_diff']  = cleaned_df['p1_reach']  - cleaned_df['p2_reach']
cleaned_df['weight_diff'] = cleaned_df['p1_weight'] - cleaned_df['p2_weight']
cleaned_df['age_diff']    = cleaned_df['p1_age_at_event'] - cleaned_df['p2_age_at_event']

#compute fighting metric differences
metrics = ['SLpM', 'Str. Acc.', 'SApM', 'Str. Def', 'TD Avg.', 'TD Acc.', 'TD Def.', 'Sub. Avg.']

for m in metrics:
  p1_col = f'p1_{m}'
  p2_col = f'p2_{m}'
  diff_col = f'{m.replace(" ", "").replace(".", "").lower()}_diff'
  cleaned_df[diff_col] = cleaned_df[p1_col] - cleaned_df[p2_col]


#find  days since last fight
from collections import defaultdict

def calculate_days_since_last_fight(df):
    df = df.sort_values('event_date').reset_index(drop=True)
    fighter_dates = defaultdict(list)
    
    # Pre-process all fight dates per fighter
    for _, row in df.iterrows():
        fighter_dates[row['p1_fighter']].append(row['event_date'])
        fighter_dates[row['p2_fighter']].append(row['event_date'])
    
    # Find previous fight dates
    def get_prev_date(fighter, current_date):
        dates = [d for d in fighter_dates[fighter] if d < current_date]
        return max(dates) if dates else None
    
    df['p1_prev_date'] = df.apply(lambda x: get_prev_date(x['p1_fighter'], x['event_date']), axis=1)
    df['p2_prev_date'] = df.apply(lambda x: get_prev_date(x['p2_fighter'], x['event_date']), axis=1)
    
    # Calculate day differences
    df['p1_days_since_last_fight'] = (df['event_date'] - df['p1_prev_date']).dt.days
    df['p2_days_since_last_fight'] = (df['event_date'] - df['p2_prev_date']).dt.days
    
    # Calculate the difference (similar to your other diff metrics)
    df['days_since_last_fight_diff'] = df['p1_days_since_last_fight'] - df['p2_days_since_last_fight']
    
    # Drop temporary columns if not needed
    df = df.drop(columns=['p1_prev_date', 'p2_prev_date'])
    
    return df

cleaned_df = calculate_days_since_last_fight(cleaned_df)
  
  
# Sort the dataframe by event_date to ensure chronological order
cleaned_df = cleaned_df.sort_values(by='event_date')

# Initialize fighter record tracker
fighter_records = {}

# Create new columns
cleaned_df['p1_wins'] = 0
cleaned_df['p1_losses'] = 0
cleaned_df['p1_total'] = 0
cleaned_df['p2_wins'] = 0
cleaned_df['p2_losses'] = 0
cleaned_df['p2_total'] = 0

for index, row in cleaned_df.iterrows(): # calculate each fighters recoord relative to when the match happened
    p1 = row['p1_fighter']
    p2 = row['p2_fighter']
    winner = row['winner']
    
    # Get current records (wins, losses, total fights)
    p1_record = fighter_records.get(p1, (0, 0, 0))
    p2_record = fighter_records.get(p2, (0, 0, 0))
    
    # Assign pre-fight records
    cleaned_df.at[index, 'p1_wins'] = p1_record[0]
    cleaned_df.at[index, 'p1_losses'] = p1_record[1]
    cleaned_df.at[index, 'p1_total'] = p1_record[0] + p1_record[1]
    
    cleaned_df.at[index, 'p2_wins'] = p2_record[0]
    cleaned_df.at[index, 'p2_losses'] = p2_record[1]
    cleaned_df.at[index, 'p2_total'] = p2_record[0] + p2_record[1]
    
    # Update records after fight based on the winner
    if winner == 1:  # p1 won
        fighter_records[p1] = (p1_record[0] + 1, p1_record[1], p1_record[0] + p1_record[1] + 1)
        fighter_records[p2] = (p2_record[0], p2_record[1] + 1, p2_record[0] + p2_record[1] + 1)
    else:  # p2 won
        fighter_records[p1] = (p1_record[0], p1_record[1] + 1, p1_record[0] + p1_record[1] + 1)
        fighter_records[p2] = (p2_record[0] + 1, p2_record[1], p2_record[0] + p2_record[1] + 1)

# Calculate differences
cleaned_df['win_diff'] = cleaned_df['p1_wins'] - cleaned_df['p2_wins']
cleaned_df['loss_diff'] = cleaned_df['p1_losses'] - cleaned_df['p2_losses']
cleaned_df['total_diff'] = cleaned_df['p1_total'] - cleaned_df['p2_total']


# Initialize columns
cleaned_df['p1_win_streak'] = 0
cleaned_df['p2_win_streak'] = 0

# Initialize win streak tracker
win_streaks = {}

# Calculate win streaks relative to when match happened
for idx, row in cleaned_df.iterrows():
    p1, p2 = row['p1_fighter'], row['p2_fighter']
    winner = row['winner']
    
    # Assign pre-fight streaks
    cleaned_df.at[idx, 'p1_win_streak'] = win_streaks.get(p1, 0)
    cleaned_df.at[idx, 'p2_win_streak'] = win_streaks.get(p2, 0)
    
    # Update streaks post-fight based on actual winner
    if winner == 1:  # p1 won
        win_streaks[p1] = win_streaks.get(p1, 0) + 1
        win_streaks[p2] = 0  # Reset losing fighter's streak
    else:  # p2 won
        win_streaks[p1] = 0  # Reset losing fighter's streak
        win_streaks[p2] = win_streaks.get(p2, 0) + 1

# Define columns to process for both fighters
p1_cols = ['p1_SLpM','p1_Str. Acc.','p1_SApM','p1_Str. Def','p1_TD Avg.','p1_TD Acc.','p1_TD Def.','p1_Sub. Avg.']
p2_cols = ['p2_SLpM','p2_Str. Acc.','p2_SApM','p2_Str. Def','p2_TD Avg.','p2_TD Acc.','p2_TD Def.','p2_Sub. Avg.']

# Create age-adjusted versions for all specified columns
for p1_col, p2_col in zip(p1_cols, p2_cols):
    cleaned_df[p1_col.replace('p1_', 'p1_age_adjusted_')] = cleaned_df[p1_col] * (1/cleaned_df['p1_age_at_event'])
    cleaned_df[p2_col.replace('p2_', 'p2_age_adjusted_')] = cleaned_df[p2_col] * (1/cleaned_df['p2_age_at_event'])

cleaned_df.head()

,p1_fighter,p2_fighter,p1_KD,p2_KD,p1_SIG_STR_PCT,p2_SIG_STR_PCT,p1_TD_PCT,p2_TD_PCT,p1_SUB_ATT,p2_SUB_ATT,p1_REV,p2_REV,p1_CTRL,p2_CTRL,p1_R1_KD,p2_R1_KD,p1_R1_SIG_STR_PCT,p2_R1_SIG_STR_PCT,p1_R1_TD_PCT,p2_R1_TD_PCT,p1_R1_SUB_ATT,p2_R1_SUB_ATT,p1_R1_REV,p2_R1_REV,p1_R1_CTRL,p2_R1_CTRL,p1_SIG_STR_PCT_DETAILED,p2_SIG_STR_PCT_DETAILED,p1_R1_SIG_STR_PCT_DETAILED,p2_R1_SIG_STR_PCT_DETAILED,method,referee,winner,event_date,p1_SIG_STR_LANDED,p1_SIG_STR_ATTEMPTED,p2_SIG_STR_LANDED,p2_SIG_STR_ATTEMPTED,p1_TOTAL_STR_LANDED,p1_TOTAL_STR_ATTEMPTED,p2_TOTAL_STR_LANDED,p2_TOTAL_STR_ATTEMPTED,p1_TD_LANDED,p1_TD_ATTEMPTED,p2_TD_LANDED,p2_TD_ATTEMPTED,p1_R1_SIG_STR_LANDED,p1_R1_SIG_STR_ATTEMPTED,p2_R1_SIG_STR_LANDED,p2_R1_SIG_STR_ATTEMPTED,p1_R1_TOTAL_STR_LANDED,p1_R1_TOTAL_STR_ATTEMPTED,p2_R1_TOTAL_STR_LANDED,p2_R1_TOTAL_STR_ATTEMPTED,p1_R1_TD_LANDED,p1_R1_TD_ATTEMPTED,p2_R1_TD_LANDED,p2_R1_TD_ATTEMPTED,p1_HEAD_LANDED,p1_HEAD_ATTEMPTED,p2_HEAD_LANDED,p2_HEAD_ATTEMPTED,p1_BODY_LANDED,p1_BODY_ATTEMPTED,p2_BODY_LANDED,p2_BODY_ATTEMPTED,p1_LEG_LANDED,p1_LEG_ATTEMPTED,p2_LEG_LANDED,p2_LEG_ATTEMPTED,p1_DISTANCE_LANDED,p1_DISTANCE_ATTEMPTED,p2_DISTANCE_LANDED,p2_DISTANCE_ATTEMPTED,p1_CLINCH_LANDED,p1_CLINCH_ATTEMPTED,p2_CLINCH_LANDED,p2_CLINCH_ATTEMPTED,p1_GROUND_LANDED,p1_GROUND_ATTEMPTED,p2_GROUND_LANDED,p2_GROUND_ATTEMPTED,p1_R1_HEAD_LANDED,p1_R1_HEAD_ATTEMPTED,p2_R1_HEAD_LANDED,p2_R1_HEAD_ATTEMPTED,p1_R1_BODY_LANDED,p1_R1_BODY_ATTEMPTED,p2_R1_BODY_LANDED,p2_R1_BODY_ATTEMPTED,p1_R1_LEG_LANDED,p1_R1_LEG_ATTEMPTED,p2_R1_LEG_LANDED,p2_R1_LEG_ATTEMPTED,p1_R1_DISTANCE_LANDED,p1_R1_DISTANCE_ATTEMPTED,p2_R1_DISTANCE_LANDED,p2_R1_DISTANCE_ATTEMPTED,p1_R1_CLINCH_LANDED,p1_R1_CLINCH_ATTEMPTED,p2_R1_CLINCH_LANDED,p2_R1_CLINCH_ATTEMPTED,p1_R1_GROUND_LANDED,p1_R1_GROUND_ATTEMPTED,p2_R1_GROUND_LANDED,p2_R1_GROUND_ATTEMPTED,p1_height,p1_weight,p1_reach,p1_stance,p1_SLpM,p1_Str. Acc.,p1_SApM,p1_Str. Def,p1_TD Avg.,p1_TD Acc.,p1_TD Def.,p1_Sub. Avg.,p2_height,p2_weight,p2_reach,p2_stance,p2_SLpM,p2_Str. Acc.,p2_SApM,p2_Str. Def,p2_TD Avg.,p2_TD Acc.,p2_TD Def.,p2_Sub. Avg.,p1_age_at_event,p2_age_at_event,height_diff,reach_diff,weight_diff,age_diff,slpm_diff,stracc_diff,sapm_diff,strdef_diff,tdavg_diff,tdacc_diff,tddef_diff,subavg_diff,p1_days_since_last_fight,p2_days_since_last_fight,days_since_last_fight_diff,p1_wins,p1_losses,p1_total,p2_wins,p2_losses,p2_total,win_diff,loss_diff,total_diff,p1_win_streak,p2_win_streak,p1_age_adjusted_SLpM,p2_age_adjusted_SLpM,p1_age_adjusted_Str. Acc.,p2_age_adjusted_Str. Acc.,p1_age_adjusted_SApM,p2_age_adjusted_SApM,p1_age_adjusted_Str. Def,p2_age_adjusted_Str. Def,p1_age_adjusted_TD Avg.,p2_age_adjusted_TD Avg.,p1_age_adjusted_TD Acc.,p2_age_adjusted_TD Acc.,p1_age_adjusted_TD Def.,p2_age_adjusted_TD Def.,p1_age_adjusted_Sub. Avg.,p2_age_adjusted_Sub. Avg.
0,Scott Morris,Sean Daugherty,0,0,1.00,0.00,1.0,NaN,1,0,0,0,0,0,0,0,1.00,0.00,1.0,NaN,1,0,0,0,0,0,1.00,0.00,1.00,0.00,Submission,John McCarthy,1,1994-03-11,1,1,0,4,2,2,1,5,1,1,0,0,1,1,0,4,2,2,1,5,1,1,0,0,1,1,0,2,0,0,0,0,0,0,0,2,0,0,0,3,1,1,0,1,0,0,0,0,1,1,0,2,0,0,0,0,0,0,0,2,0,0,0,3,1,1,0,1,0,0,0,0,70.0,210.0,NaN,Orthodox,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72.0,175.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,18.26694,-2.0,NaN,35.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
14,Patrick Smith,Scott Morris,0,0,0.76,NaN,NaN,0.0,0,0,0,0,0,0,0,0,0.76,NaN,NaN,0.0,0,0,0,0,0,0,0.76,NaN,0.76,NaN,KO/TKO,John McCarthy,1,1994-03-11,13,17,0,0,19,25,0,0,0,0,0,1,13,17,0,0,19,25,0,0,0,0,0,1,12,16,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,12,16,0,0,12,16,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,12,16,0,0,74.0,225.0,NaN,Orthodox,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.0,210.0,NaN,Orthodox,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.53525,NaN,4.0,NaN,15.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0,0,0,1,0,1,-1,0,-1,0,1,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN
13,Johnny Rhodes,David Levicki,0,0,0.64,

In [3]:
#calculate all the ema

# Ensure data is sorted by event_date
df = cleaned_df.sort_values(by='event_date')

# List of features to calculate EMA for
features = [
    'KD', 'SIG_STR_PCT', 'TD_PCT', 'SUB_ATT', 'REV', 'CTRL',
    'R1_KD', 'R1_SIG_STR_PCT', 'R1_TD_PCT', 'R1_SUB_ATT', 'R1_REV', 'R1_CTRL',
    'SIG_STR_PCT_DETAILED', 'R1_SIG_STR_PCT_DETAILED',
    'SIG_STR_LANDED', 'SIG_STR_ATTEMPTED', 'TOTAL_STR_LANDED', 'TOTAL_STR_ATTEMPTED',
    'TD_LANDED', 'TD_ATTEMPTED',
    'R1_SIG_STR_LANDED', 'R1_SIG_STR_ATTEMPTED', 'R1_TOTAL_STR_LANDED', 'R1_TOTAL_STR_ATTEMPTED',
    'R1_TD_LANDED', 'R1_TD_ATTEMPTED',
    'HEAD_LANDED', 'HEAD_ATTEMPTED', 'BODY_LANDED', 'BODY_ATTEMPTED',
    'LEG_LANDED', 'LEG_ATTEMPTED',
    'DISTANCE_LANDED', 'DISTANCE_ATTEMPTED', 'CLINCH_LANDED', 'CLINCH_ATTEMPTED',
    'GROUND_LANDED', 'GROUND_ATTEMPTED',
    'R1_HEAD_LANDED', 'R1_HEAD_ATTEMPTED', 'R1_BODY_LANDED', 'R1_BODY_ATTEMPTED',
    'R1_LEG_LANDED', 'R1_LEG_ATTEMPTED',
    'R1_DISTANCE_LANDED', 'R1_DISTANCE_ATTEMPTED', 'R1_CLINCH_LANDED', 'R1_CLINCH_ATTEMPTED',
    'R1_GROUND_LANDED', 'R1_GROUND_ATTEMPTED'
]

# Process p1 fighter data
p1_cols = {f'p1_{feat}': feat for feat in features}
p1_stats = df[['p1_fighter', 'p2_fighter', 'event_date'] + list(p1_cols.keys())].copy()
p1_stats['fighter'] = p1_stats['p1_fighter']
p1_stats['position'] = 'p1'
p1_stats = p1_stats.rename(columns=p1_cols)

# Process p2 fighter data
p2_cols = {f'p2_{feat}': feat for feat in features}
p2_stats = df[['p1_fighter', 'p2_fighter', 'event_date'] + list(p2_cols.keys())].copy()
p2_stats['fighter'] = p2_stats['p2_fighter']
p2_stats['position'] = 'p2'
p2_stats = p2_stats.rename(columns=p2_cols)

# Combine fighter stats
columns_to_keep = ['fighter', 'position', 'event_date', 'p1_fighter', 'p2_fighter'] + list(features)
fighter_stats = pd.concat([p1_stats[columns_to_keep], p2_stats[columns_to_keep]], ignore_index=True)

# Sort by fighter and date
fighter_stats = fighter_stats.sort_values(by=['fighter', 'event_date']).copy()

# Convert all feature columns to numeric
for feat in features:
    fighter_stats[feat] = pd.to_numeric(fighter_stats[feat], errors='coerce')

# Calculate EMAs for all features grouped by fighter
ema_dfs = []
for feat in features:
    ema_series = fighter_stats.groupby('fighter')[feat].transform(
        lambda x: x.ewm(span=3).mean()
    )
    ema_dfs.append(pd.DataFrame({f'{feat}_EMA': ema_series}))

# Join all EMAs at once
ema_df = pd.concat(ema_dfs, axis=1)
fighter_stats = pd.concat([fighter_stats, ema_df], axis=1)

# Create final result DataFrame
result_df = df.copy()

# Initialize EMA columns in the result DataFrame at once
ema_columns = {}
for feat in features:
    ema_columns[f'p1_{feat}_EMA'] = np.nan
    ema_columns[f'p2_{feat}_EMA'] = np.nan

result_ema_df = pd.DataFrame(ema_columns, index=result_df.index)
result_df = pd.concat([result_df, result_ema_df], axis=1)

# Add EMA values to result DataFrame
for idx, row in result_df.iterrows():
    date = row['event_date']
    
    # Get p1 fighter EMA values from previous fights
    p1_fighter = row['p1_fighter']
    p1_prev = fighter_stats[(fighter_stats['fighter'] == p1_fighter) & 
                           (fighter_stats['event_date'] < date)]
    
    if not p1_prev.empty:
        p1_latest = p1_prev.sort_values('event_date').iloc[-1]
        for feat in features:
            ema_col = f'{feat}_EMA'
            if ema_col in p1_latest and not pd.isna(p1_latest[ema_col]):
                result_df.loc[idx, f'p1_{feat}_EMA'] = p1_latest[ema_col]
    
    # Get p2 fighter EMA values from previous fights
    p2_fighter = row['p2_fighter']
    p2_prev = fighter_stats[(fighter_stats['fighter'] == p2_fighter) & 
                           (fighter_stats['event_date'] < date)]
    
    if not p2_prev.empty:
        p2_latest = p2_prev.sort_values('event_date').iloc[-1]
        for feat in features:
            ema_col = f'{feat}_EMA'
            if ema_col in p2_latest and not pd.isna(p2_latest[ema_col]):
                result_df.loc[idx, f'p2_{feat}_EMA'] = p2_latest[ema_col]

# Defragment the final DataFrame
result_df = result_df.copy()


In [4]:
methods_df = result_df.copy()

# Create a copy of the 'method' column to avoid modifying the original directly
methods_df['method_grouped'] = methods_df['method'].copy()

# Group the 'Decision' types
methods_df['method_grouped'] = methods_df['method_grouped'].replace(
    {'Decision - Majority': 'Decision',
     'Decision - Split': 'Decision',
     'Decision - Unanimous': 'Decision'}
)
# Group the 'TKO/KO' types
methods_df['method_grouped'] = methods_df['method_grouped'].replace(
    {"TKO - Doctor's Stoppage": "KO/TKO"}
)

# Group the remaining into 'Other'
other_categories = ['Overturned', 'Could Not Continue', 'DQ', 'Other']
methods_df['method_grouped'] = methods_df['method_grouped'].replace(
    {category: 'Other' for category in other_categories}
)

# Sort by event date to ensure chronological order
methods_df = methods_df.sort_values('event_date').reset_index(drop=True)

# Initialize dictionaries to track cumulative wins by method for each fighter
method_counts = {
    'Decision': defaultdict(int),
    'KO/TKO': defaultdict(int),
    'Submission': defaultdict(int),
    'Other': defaultdict(int)
}

# Create empty columns for our counts
for method in ['Decision', 'KO/TKO', 'Submission', 'Other']:
    methods_df[f'p1_{method}_wins'] = 0
    methods_df[f'p2_{method}_wins'] = 0

# Iterate through each fight chronologically
for idx, row in methods_df.iterrows():
    current_method = row['method_grouped']
    winner = row['winner']
    p1 = row['p1_fighter']
    p2 = row['p2_fighter']
    
    # Assign current counts BEFORE updating for this fight's result
    for method in method_counts.keys():
        methods_df.at[idx, f'p1_{method}_wins'] = method_counts[method][p1]
        methods_df.at[idx, f'p2_{method}_wins'] = method_counts[method][p2]
    
    # Update counts AFTER assignment (to avoid including current fight)
    if winner == 1:  # p1 won
        method_counts[current_method][p1] += 1
    else:  # p2 won
        method_counts[current_method][p2] += 1

# Show the results
methods_df.head()

,p1_fighter,p2_fighter,p1_KD,p2_KD,p1_SIG_STR_PCT,p2_SIG_STR_PCT,p1_TD_PCT,p2_TD_PCT,p1_SUB_ATT,p2_SUB_ATT,p1_REV,p2_REV,p1_CTRL,p2_CTRL,p1_R1_KD,p2_R1_KD,p1_R1_SIG_STR_PCT,p2_R1_SIG_STR_PCT,p1_R1_TD_PCT,p2_R1_TD_PCT,p1_R1_SUB_ATT,p2_R1_SUB_ATT,p1_R1_REV,p2_R1_REV,p1_R1_CTRL,p2_R1_CTRL,p1_SIG_STR_PCT_DETAILED,p2_SIG_STR_PCT_DETAILED,p1_R1_SIG_STR_PCT_DETAILED,p2_R1_SIG_STR_PCT_DETAILED,method,referee,winner,event_date,p1_SIG_STR_LANDED,p1_SIG_STR_ATTEMPTED,p2_SIG_STR_LANDED,p2_SIG_STR_ATTEMPTED,p1_TOTAL_STR_LANDED,p1_TOTAL_STR_ATTEMPTED,p2_TOTAL_STR_LANDED,p2_TOTAL_STR_ATTEMPTED,p1_TD_LANDED,p1_TD_ATTEMPTED,p2_TD_LANDED,p2_TD_ATTEMPTED,p1_R1_SIG_STR_LANDED,p1_R1_SIG_STR_ATTEMPTED,p2_R1_SIG_STR_LANDED,p2_R1_SIG_STR_ATTEMPTED,p1_R1_TOTAL_STR_LANDED,p1_R1_TOTAL_STR_ATTEMPTED,p2_R1_TOTAL_STR_LANDED,p2_R1_TOTAL_STR_ATTEMPTED,p1_R1_TD_LANDED,p1_R1_TD_ATTEMPTED,p2_R1_TD_LANDED,p2_R1_TD_ATTEMPTED,p1_HEAD_LANDED,p1_HEAD_ATTEMPTED,p2_HEAD_LANDED,p2_HEAD_ATTEMPTED,p1_BODY_LANDED,p1_BODY_ATTEMPTED,p2_BODY_LANDED,p2_BODY_ATTEMPTED,p1_LEG_LANDED,p1_LEG_ATTEMPTED,p2_LEG_LANDED,p2_LEG_ATTEMPTED,p1_DISTANCE_LANDED,p1_DISTANCE_ATTEMPTED,p2_DISTANCE_LANDED,p2_DISTANCE_ATTEMPTED,p1_CLINCH_LANDED,p1_CLINCH_ATTEMPTED,p2_CLINCH_LANDED,p2_CLINCH_ATTEMPTED,p1_GROUND_LANDED,p1_GROUND_ATTEMPTED,p2_GROUND_LANDED,p2_GROUND_ATTEMPTED,p1_R1_HEAD_LANDED,p1_R1_HEAD_ATTEMPTED,p2_R1_HEAD_LANDED,p2_R1_HEAD_ATTEMPTED,p1_R1_BODY_LANDED,p1_R1_BODY_ATTEMPTED,p2_R1_BODY_LANDED,p2_R1_BODY_ATTEMPTED,p1_R1_LEG_LANDED,p1_R1_LEG_ATTEMPTED,p2_R1_LEG_LANDED,p2_R1_LEG_ATTEMPTED,p1_R1_DISTANCE_LANDED,p1_R1_DISTANCE_ATTEMPTED,p2_R1_DISTANCE_LANDED,p2_R1_DISTANCE_ATTEMPTED,p1_R1_CLINCH_LANDED,p1_R1_CLINCH_ATTEMPTED,p2_R1_CLINCH_LANDED,p2_R1_CLINCH_ATTEMPTED,p1_R1_GROUND_LANDED,p1_R1_GROUND_ATTEMPTED,p2_R1_GROUND_LANDED,p2_R1_GROUND_ATTEMPTED,p1_height,p1_weight,p1_reach,p1_stance,p1_SLpM,p1_Str. Acc.,p1_SApM,p1_Str. Def,p1_TD Avg.,p1_TD Acc.,p1_TD Def.,p1_Sub. Avg.,p2_height,p2_weight,p2_reach,p2_stance,p2_SLpM,p2_Str. Acc.,p2_SApM,p2_Str. Def,p2_TD Avg.,p2_TD Acc.,p2_TD Def.,p2_Sub. Avg.,p1_age_at_event,p2_age_at_event,height_diff,reach_diff,weight_diff,age_diff,slpm_diff,stracc_diff,sapm_diff,strdef_diff,tdavg_diff,tdacc_diff,tddef_diff,subavg_diff,p1_days_since_last_fight,p2_days_since_last_fight,days_since_last_fight_diff,p1_wins,p1_losses,p1_total,p2_wins,p2_losses,p2_total,win_diff,loss_diff,total_diff,p1_win_streak,p2_win_streak,p1_age_adjusted_SLpM,p2_age_adjusted_SLpM,p1_age_adjusted_Str. Acc.,p2_age_adjusted_Str. Acc.,p1_age_adjusted_SApM,p2_age_adjusted_SApM,p1_age_adjusted_Str. Def,p2_age_adjusted_Str. Def,p1_age_adjusted_TD Avg.,p2_age_adjusted_TD Avg.,p1_age_adjusted_TD Acc.,p2_age_adjusted_TD Acc.,p1_age_adjusted_TD Def.,p2_age_adjusted_TD Def.,p1_age_adjusted_Sub. Avg.,p2_age_adjusted_Sub. Avg.,p1_KD_EMA,p2_KD_EMA,p1_SIG_STR_PCT_EMA,p2_SIG_STR_PCT_EMA,p1_TD_PCT_EMA,p2_TD_PCT_EMA,p1_SUB_ATT_EMA,p2_SUB_ATT_EMA,p1_REV_EMA,p2_REV_EMA,p1_CTRL_EMA,p2_CTRL_EMA,p1_R1_KD_EMA,p2_R1_KD_EMA,p1_R1_SIG_STR_PCT_EMA,p2_R1_SIG_STR_PCT_EMA,p1_R1_TD_PCT_EMA,p2_R1_TD_PCT_EMA,p1_R1_SUB_ATT_EMA,p2_R1_SUB_ATT_EMA,p1_R1_REV_EMA,p2_R1_REV_EMA,p1_R1_CTRL_EMA,p2_R1_CTRL_EMA,p1_SIG_STR_PCT_DETAILED_EMA,p2_SIG_STR_PCT_DETAILED_EMA,p1_R1_SIG_STR_PCT_DETAILED_EMA,p2_R1_SIG_STR_PCT_DETAILED_EMA,p1_SIG_STR_LANDED_EMA,p2_SIG_STR_LANDED_EMA,p1_SIG_STR_ATTEMPTED_EMA,p2_SIG_STR_ATTEMPTED_EMA,p1_TOTAL_STR_LANDED_EMA,p2_TOTAL_STR_LANDED_EMA,p1_TOTAL_STR_ATTEMPTED_EMA,p2_TOTAL_STR_ATTEMPTED_EMA,p1_TD_LANDED_EMA,p2_TD_LANDED_EMA,p1_TD_ATTEMPTED_EMA,p2_TD_ATTEMPTED_EMA,p1_R1_SIG_STR_LANDED_EMA,p2_R1_SIG_STR_LANDED_EMA,p1_R1_SIG_STR_ATTEMPTED_EMA,p2_R1_SIG_STR_ATTEMPTED_EMA,p1_R1_TOTAL_STR_LANDED_EMA,p2_R1_TOTAL_STR_LANDED_EMA,p1_R1_TOTAL_STR_ATTEMPTED_EMA,p2_R1_TOTAL_STR_ATTEMPTED_EMA,p1_R1_TD_LANDED_EMA,p2_R1_TD_LANDED_EMA,p1_R1_TD_ATTEMPTED_EMA,p2_R1_TD_ATTEMPTED_EMA,p1_HEAD_LANDED_EMA,p2_HEAD_LANDED_EMA,p1_HEAD_ATTEMPTED_EMA,p2_HEAD_ATTEMPTED_EMA,p1_BODY_LANDED_EMA,p2_BODY_LANDED_EMA,p1_BODY_ATTEMPTE

In [5]:
# Drop all in match data

result_df.sort_values(by='event_date', ascending=False, inplace=True)
columns_to_drop = [
    'p1_KD', 'p2_KD', 'p1_SIG_STR_PCT', 'p2_SIG_STR_PCT', 'p1_TD_PCT', 'p2_TD_PCT', 
    'p1_SUB_ATT', 'p2_SUB_ATT', 'p1_REV', 'p2_REV', 'p1_CTRL', 'p2_CTRL', 
    'p1_R1_KD', 'p2_R1_KD', 'p1_R1_SIG_STR_PCT', 'p2_R1_SIG_STR_PCT', 
    'p1_R1_TD_PCT', 'p2_R1_TD_PCT', 'p1_R1_SUB_ATT', 'p2_R1_SUB_ATT', 
    'p1_R1_REV', 'p2_R1_REV', 'p1_R1_CTRL', 'p2_R1_CTRL', 
    'p1_SIG_STR_PCT_DETAILED', 'p2_SIG_STR_PCT_DETAILED', 
    'p1_R1_SIG_STR_PCT_DETAILED', 'p2_R1_SIG_STR_PCT_DETAILED',
    'p1_SIG_STR_LANDED', 'p1_SIG_STR_ATTEMPTED', 'p2_SIG_STR_LANDED', 'p2_SIG_STR_ATTEMPTED',
    'p1_TOTAL_STR_LANDED', 'p1_TOTAL_STR_ATTEMPTED', 'p2_TOTAL_STR_LANDED', 'p2_TOTAL_STR_ATTEMPTED',
    'p1_TD_LANDED', 'p1_TD_ATTEMPTED', 'p2_TD_LANDED', 'p2_TD_ATTEMPTED',
    'p1_R1_SIG_STR_LANDED', 'p1_R1_SIG_STR_ATTEMPTED', 'p2_R1_SIG_STR_LANDED', 'p2_R1_SIG_STR_ATTEMPTED',
    'p1_R1_TOTAL_STR_LANDED', 'p1_R1_TOTAL_STR_ATTEMPTED', 'p2_R1_TOTAL_STR_LANDED', 'p2_R1_TOTAL_STR_ATTEMPTED',
    'p1_R1_TD_LANDED', 'p1_R1_TD_ATTEMPTED', 'p2_R1_TD_LANDED', 'p2_R1_TD_ATTEMPTED',
    'p1_HEAD_LANDED', 'p1_HEAD_ATTEMPTED', 'p2_HEAD_LANDED', 'p2_HEAD_ATTEMPTED',
    'p1_BODY_LANDED', 'p1_BODY_ATTEMPTED', 'p2_BODY_LANDED', 'p2_BODY_ATTEMPTED',
    'p1_LEG_LANDED', 'p1_LEG_ATTEMPTED', 'p2_LEG_LANDED', 'p2_LEG_ATTEMPTED',
    'p1_DISTANCE_LANDED', 'p1_DISTANCE_ATTEMPTED', 'p2_DISTANCE_LANDED', 'p2_DISTANCE_ATTEMPTED',
    'p1_CLINCH_LANDED', 'p1_CLINCH_ATTEMPTED', 'p2_CLINCH_LANDED', 'p2_CLINCH_ATTEMPTED',
    'p1_GROUND_LANDED', 'p1_GROUND_ATTEMPTED', 'p2_GROUND_LANDED', 'p2_GROUND_ATTEMPTED',
    'p1_R1_HEAD_LANDED', 'p1_R1_HEAD_ATTEMPTED', 'p2_R1_HEAD_LANDED', 'p2_R1_HEAD_ATTEMPTED',
    'p1_R1_BODY_LANDED', 'p1_R1_BODY_ATTEMPTED', 'p2_R1_BODY_LANDED', 'p2_R1_BODY_ATTEMPTED',
    'p1_R1_LEG_LANDED', 'p1_R1_LEG_ATTEMPTED', 'p2_R1_LEG_LANDED', 'p2_R1_LEG_ATTEMPTED',
    'p1_R1_DISTANCE_LANDED', 'p1_R1_DISTANCE_ATTEMPTED', 'p2_R1_DISTANCE_LANDED', 'p2_R1_DISTANCE_ATTEMPTED',
    'p1_R1_CLINCH_LANDED', 'p1_R1_CLINCH_ATTEMPTED', 'p2_R1_CLINCH_LANDED', 'p2_R1_CLINCH_ATTEMPTED',
    'p1_R1_GROUND_LANDED', 'p1_R1_GROUND_ATTEMPTED', 'p2_R1_GROUND_LANDED', 'p2_R1_GROUND_ATTEMPTED',
    'method' 
] # drop event_date 

# Drop the columns
result_df = result_df.drop(columns=columns_to_drop)
methods_df = methods_df.drop(columns=columns_to_drop)

result_df.head()

,p1_fighter,p2_fighter,referee,winner,event_date,p1_height,p1_weight,p1_reach,p1_stance,p1_SLpM,p1_Str. Acc.,p1_SApM,p1_Str. Def,p1_TD Avg.,p1_TD Acc.,p1_TD Def.,p1_Sub. Avg.,p2_height,p2_weight,p2_reach,p2_stance,p2_SLpM,p2_Str. Acc.,p2_SApM,p2_Str. Def,p2_TD Avg.,p2_TD Acc.,p2_TD Def.,p2_Sub. Avg.,p1_age_at_event,p2_age_at_event,height_diff,reach_diff,weight_diff,age_diff,slpm_diff,stracc_diff,sapm_diff,strdef_diff,tdavg_diff,tdacc_diff,tddef_diff,subavg_diff,p1_days_since_last_fight,p2_days_since_last_fight,days_since_last_fight_diff,p1_wins,p1_losses,p1_total,p2_wins,p2_losses,p2_total,win_diff,loss_diff,total_diff,p1_win_streak,p2_win_streak,p1_age_adjusted_SLpM,p2_age_adjusted_SLpM,p1_age_adjusted_Str. Acc.,p2_age_adjusted_Str. Acc.,p1_age_adjusted_SApM,p2_age_adjusted_SApM,p1_age_adjusted_Str. Def,p2_age_adjusted_Str. Def,p1_age_adjusted_TD Avg.,p2_age_adjusted_TD Avg.,p1_age_adjusted_TD Acc.,p2_age_adjusted_TD Acc.,p1_age_adjusted_TD Def.,p2_age_adjusted_TD Def.,p1_age_adjusted_Sub. Avg.,p2_age_adjusted_Sub. Avg.,p1_KD_EMA,p2_KD_EMA,p1_SIG_STR_PCT_EMA,p2_SIG_STR_PCT_EMA,p1_TD_PCT_EMA,p2_TD_PCT_EMA,p1_SUB_ATT_EMA,p2_SUB_ATT_EMA,p1_REV_EMA,p2_REV_EMA,p1_CTRL_EMA,p2_CTRL_EMA,p1_R1_KD_EMA,p2_R1_KD_EMA,p1_R1_SIG_STR_PCT_EMA,p2_R1_SIG_STR_PCT_EMA,p1_R1_TD_PCT_EMA,p2_R1_TD_PCT_EMA,p1_R1_SUB_ATT_EMA,p2_R1_SUB_ATT_EMA,p1_R1_REV_EMA,p2_R1_REV_EMA,p1_R1_CTRL_EMA,p2_R1_CTRL_EMA,p1_SIG_STR_PCT_DETAILED_EMA,p2_SIG_STR_PCT_DETAILED_EMA,p1_R1_SIG_STR_PCT_DETAILED_EMA,p2_R1_SIG_STR_PCT_DETAILED_EMA,p1_SIG_STR_LANDED_EMA,p2_SIG_STR_LANDED_EMA,p1_SIG_STR_ATTEMPTED_EMA,p2_SIG_STR_ATTEMPTED_EMA,p1_TOTAL_STR_LANDED_EMA,p2_TOTAL_STR_LANDED_EMA,p1_TOTAL_STR_ATTEMPTED_EMA,p2_TOTAL_STR_ATTEMPTED_EMA,p1_TD_LANDED_EMA,p2_TD_LANDED_EMA,p1_TD_ATTEMPTED_EMA,p2_TD_ATTEMPTED_EMA,p1_R1_SIG_STR_LANDED_EMA,p2_R1_SIG_STR_LANDED_EMA,p1_R1_SIG_STR_ATTEMPTED_EMA,p2_R1_SIG_STR_ATTEMPTED_EMA,p1_R1_TOTAL_STR_LANDED_EMA,p2_R1_TOTAL_STR_LANDED_EMA,p1_R1_TOTAL_STR_ATTEMPTED_EMA,p2_R1_TOTAL_STR_ATTEMPTED_EMA,p1_R1_TD_LANDED_EMA,p2_R1_TD_LANDED_EMA,p1_R1_TD_ATTEMPTED_EMA,p2_R1_TD_ATTEMPTED_EMA,p1_HEAD_LANDED_EMA,p2_HEAD_LANDED_EMA,p1_HEAD_ATTEMPTED_EMA,p2_HEAD_ATTEMPTED_EMA,p1_BODY_LANDED_EMA,p2_BODY_LANDED_EMA,p1_BODY_ATTEMPTED_EMA,p2_BODY_ATTEMPTED_EMA,p1_LEG_LANDED_EMA,p2_LEG_LANDED_EMA,p1_LEG_ATTEMPTED_EMA,p2_LEG_ATTEMPTED_EMA,p1_DISTANCE_LANDED_EMA,p2_DISTANCE_LANDED_EMA,p1_DISTANCE_ATTEMPTED_EMA,p2_DISTANCE_ATTEMPTED_EMA,p1_CLINCH_LANDED_EMA,p2_CLINCH_LANDED_EMA,p1_CLINCH_ATTEMPTED_EMA,p2_CLINCH_ATTEMPTED_EMA,p1_GROUND_LANDED_EMA,p2_GROUND_LANDED_EMA,p1_GROUND_ATTEMPTED_EMA,p2_GROUND_ATTEMPTED_EMA,p1_R1_HEAD_LANDED_EMA,p2_R1_HEAD_LANDED_EMA,p1_R1_HEAD_ATTEMPTED_EMA,p2_R1_HEAD_ATTEMPTED_EMA,p1_R1_BODY_LANDED_EMA,p2_R1_BODY_LANDED_EMA,p1_R1_BODY_ATTEMPTED_EMA,p2_R1_BODY_ATTEMPTED_EMA,p1_R1_LEG_LANDED_EMA,p2_R1_LEG_LANDED_EMA,p1_R1_LEG_ATTEMPTED_EMA,p2_R1_LEG_ATTEMPTED_EMA,p1_R1_DISTANCE_LANDED_EMA,p2_R1_DISTANCE_LANDED_EMA,p1_R1_DISTANCE_ATTEMPTED_EMA,p2_R1_DISTANCE_ATTEMPTED_EMA,p1_R1_CLINCH_LANDED_EMA,p2_R1_CLINCH_LANDED_EMA,p1_R1_CLINCH_ATTEMPTED_EMA,p2_R1_CLINCH_ATTEMPTED_EMA,p1_R1_GROUND_LANDED_EMA,p2_R1_GROUND_LANDED_EMA,p1_R1_GROUND_ATTEMPTED_EMA,p2_R1_GROUND_ATTEMPTED_EMA
8207,Mateusz Gamrot,Ludovit Klein,Herb Dean,1,2025-05-31,70.0,155.0,70.0,Southpaw,3.23,0.50,3.22,0.58,5.25,0.35,0.90,0.1,67.0,155.0,72.0,Southpaw,3.81,0.54,3.22,0.54,1.70,0.51,0.91,0.0,34.469541,30.269678,3.0,-2.0,0.0,4.199863,-0.58,-0.04,0.00,0.04,3.55,-0.16,-0.01,0.1,287.0,245.0,42.0,7,3,10,8,2,10,-1,1,0,0,7,0.093706,0.125869,0.014506,0.017840,0.093416,0.106377,0.016826,0.017840,0.152308,0.056162,0.010154,0.016849,0.026110,0.030063,0.002901,0.000000,0.001955,0.642229,0.506540,0.513558,0.366911,0.723151,0.003910,0.000000,0.000000,0.000000,301.408602,355.013685,0.000000,0.376344,0.500567,0.488925,0.458915,0.175786,0.003910,0.000000,0.000000,0.000000,97.304985,67.633431,0.506540,0.513558,0.500567,0.488925,52.737048,50.528837,103.116325,97.365591,82.786901,93.275660,136.028348,157.535679,

In [6]:

# Identify p1 and p2 columns
p1_cols = [col for col in result_df.columns if col.startswith('p1_')]
p2_cols = [col for col in result_df.columns if col.startswith('p2_')]

# Identify difference columns
diff_cols = [col for col in result_df.columns if 'diff' in col]

# Columns that are not swapped (e.g. referee, fight_id if present, etc.)
other_cols = [col for col in result_df.columns if col not in p1_cols + p2_cols + ['winner'] + diff_cols]

# Prepare swapped dataframe
swapped_df = result_df.copy()

# Swap p1 and p2 columns
swapped_df[p1_cols] = result_df[p2_cols].values
swapped_df[p2_cols] = result_df[p1_cols].values

# Invert all diff columns
for col in diff_cols:
    swapped_df[col] = -result_df[col]

# Swap fighter names
if 'p1_fighter' in result_df.columns and 'p2_fighter' in result_df.columns:
    swapped_df['p1_fighter'] = result_df['p2_fighter']
    swapped_df['p2_fighter'] = result_df['p1_fighter']

# Adjust winner label
def swap_winner(val):
    if val == 1:
        return 0
    elif val == 0:
        return 1
    else:
        return val

swapped_df['winner'] = result_df['winner'].apply(swap_winner)

# Concatenate original and swapped dataframes
aug_df = pd.concat([result_df, swapped_df], ignore_index=True)

# Sort by event_date
aug_df.sort_values(by='event_date', inplace=True)


In [7]:
import pandas as pd

# Identify p1 and p2 columns
p1_cols = [col for col in methods_df.columns if col.startswith('p1_')]
p2_cols = [col for col in methods_df.columns if col.startswith('p2_')]

# Identify difference columns
diff_cols = [col for col in methods_df.columns if 'diff' in col]

# Columns that are not swapped (e.g. referee, fight_id if present, etc.)
other_cols = [col for col in methods_df.columns if col not in p1_cols + p2_cols + ['winner'] + diff_cols]

# Prepare swapped dataframe
swapped_df = methods_df.copy()

# Swap p1 and p2 columns
swapped_df[p1_cols] = methods_df[p2_cols].values
swapped_df[p2_cols] = methods_df[p1_cols].values

# Invert all diff columns
for col in diff_cols:
    swapped_df[col] = -methods_df[col]

# Swap fighter names
if 'p1_fighter' in methods_df.columns and 'p2_fighter' in methods_df.columns:
    swapped_df['p1_fighter'] = methods_df['p2_fighter']
    swapped_df['p2_fighter'] = methods_df['p1_fighter']

# Adjust winner label
def swap_winner(val):
    if val == 1:
        return 0
    elif val == 0:
        return 1
    else:
        return val

swapped_df['winner'] = methods_df['winner'].apply(swap_winner)

# Concatenate original and swapped dataframes
method_aug_df = pd.concat([methods_df, swapped_df], ignore_index=True)

# Sort by event_date
method_aug_df.sort_values(by='event_date', inplace=True)


In [8]:

result_df.to_csv('ufc_features.csv', index=False)
methods_df.to_csv('ufc_methods.csv', index=False)
aug_df.to_csv('ufc_aug.csv', index=False)
method_aug_df.to_csv('ufc_method_aug.csv', index=False)

In [9]:
aug_df.head(30)

,p1_fighter,p2_fighter,referee,winner,event_date,p1_height,p1_weight,p1_reach,p1_stance,p1_SLpM,p1_Str. Acc.,p1_SApM,p1_Str. Def,p1_TD Avg.,p1_TD Acc.,p1_TD Def.,p1_Sub. Avg.,p2_height,p2_weight,p2_reach,p2_stance,p2_SLpM,p2_Str. Acc.,p2_SApM,p2_Str. Def,p2_TD Avg.,p2_TD Acc.,p2_TD Def.,p2_Sub. Avg.,p1_age_at_event,p2_age_at_event,height_diff,reach_diff,weight_diff,age_diff,slpm_diff,stracc_diff,sapm_diff,strdef_diff,tdavg_diff,tdacc_diff,tddef_diff,subavg_diff,p1_days_since_last_fight,p2_days_since_last_fight,days_since_last_fight_diff,p1_wins,p1_losses,p1_total,p2_wins,p2_losses,p2_total,win_diff,loss_diff,total_diff,p1_win_streak,p2_win_streak,p1_age_adjusted_SLpM,p2_age_adjusted_SLpM,p1_age_adjusted_Str. Acc.,p2_age_adjusted_Str. Acc.,p1_age_adjusted_SApM,p2_age_adjusted_SApM,p1_age_adjusted_Str. Def,p2_age_adjusted_Str. Def,p1_age_adjusted_TD Avg.,p2_age_adjusted_TD Avg.,p1_age_adjusted_TD Acc.,p2_age_adjusted_TD Acc.,p1_age_adjusted_TD Def.,p2_age_adjusted_TD Def.,p1_age_adjusted_Sub. Avg.,p2_age_adjusted_Sub. Avg.,p1_KD_EMA,p2_KD_EMA,p1_SIG_STR_PCT_EMA,p2_SIG_STR_PCT_EMA,p1_TD_PCT_EMA,p2_TD_PCT_EMA,p1_SUB_ATT_EMA,p2_SUB_ATT_EMA,p1_REV_EMA,p2_REV_EMA,p1_CTRL_EMA,p2_CTRL_EMA,p1_R1_KD_EMA,p2_R1_KD_EMA,p1_R1_SIG_STR_PCT_EMA,p2_R1_SIG_STR_PCT_EMA,p1_R1_TD_PCT_EMA,p2_R1_TD_PCT_EMA,p1_R1_SUB_ATT_EMA,p2_R1_SUB_ATT_EMA,p1_R1_REV_EMA,p2_R1_REV_EMA,p1_R1_CTRL_EMA,p2_R1_CTRL_EMA,p1_SIG_STR_PCT_DETAILED_EMA,p2_SIG_STR_PCT_DETAILED_EMA,p1_R1_SIG_STR_PCT_DETAILED_EMA,p2_R1_SIG_STR_PCT_DETAILED_EMA,p1_SIG_STR_LANDED_EMA,p2_SIG_STR_LANDED_EMA,p1_SIG_STR_ATTEMPTED_EMA,p2_SIG_STR_ATTEMPTED_EMA,p1_TOTAL_STR_LANDED_EMA,p2_TOTAL_STR_LANDED_EMA,p1_TOTAL_STR_ATTEMPTED_EMA,p2_TOTAL_STR_ATTEMPTED_EMA,p1_TD_LANDED_EMA,p2_TD_LANDED_EMA,p1_TD_ATTEMPTED_EMA,p2_TD_ATTEMPTED_EMA,p1_R1_SIG_STR_LANDED_EMA,p2_R1_SIG_STR_LANDED_EMA,p1_R1_SIG_STR_ATTEMPTED_EMA,p2_R1_SIG_STR_ATTEMPTED_EMA,p1_R1_TOTAL_STR_LANDED_EMA,p2_R1_TOTAL_STR_LANDED_EMA,p1_R1_TOTAL_STR_ATTEMPTED_EMA,p2_R1_TOTAL_STR_ATTEMPTED_EMA,p1_R1_TD_LANDED_EMA,p2_R1_TD_LANDED_EMA,p1_R1_TD_ATTEMPTED_EMA,p2_R1_TD_ATTEMPTED_EMA,p1_HEAD_LANDED_EMA,p2_HEAD_LANDED_EMA,p1_HEAD_ATTEMPTED_EMA,p2_HEAD_ATTEMPTED_EMA,p1_BODY_LANDED_EMA,p2_BODY_LANDED_EMA,p1_BODY_ATTEMPTED_EMA,p2_BODY_ATTEMPTED_EMA,p1_LEG_LANDED_EMA,p2_LEG_LANDED_EMA,p1_LEG_ATTEMPTED_EMA,p2_LEG_ATTEMPTED_EMA,p1_DISTANCE_LANDED_EMA,p2_DISTANCE_LANDED_EMA,p1_DISTANCE_ATTEMPTED_EMA,p2_DISTANCE_ATTEMPTED_EMA,p1_CLINCH_LANDED_EMA,p2_CLINCH_LANDED_EMA,p1_CLINCH_ATTEMPTED_EMA,p2_CLINCH_ATTEMPTED_EMA,p1_GROUND_LANDED_EMA,p2_GROUND_LANDED_EMA,p1_GROUND_ATTEMPTED_EMA,p2_GROUND_ATTEMPTED_EMA,p1_R1_HEAD_LANDED_EMA,p2_R1_HEAD_LANDED_EMA,p1_R1_HEAD_ATTEMPTED_EMA,p2_R1_HEAD_ATTEMPTED_EMA,p1_R1_BODY_LANDED_EMA,p2_R1_BODY_LANDED_EMA,p1_R1_BODY_ATTEMPTED_EMA,p2_R1_BODY_ATTEMPTED_EMA,p1_R1_LEG_LANDED_EMA,p2_R1_LEG_LANDED_EMA,p1_R1_LEG_ATTEMPTED_EMA,p2_R1_LEG_ATTEMPTED_EMA,p1_R1_DISTANCE_LANDED_EMA,p2_R1_DISTANCE_LANDED_EMA,p1_R1_DISTANCE_ATTEMPTED_EMA,p2_R1_DISTANCE_ATTEMPTED_EMA,p1_R1_CLINCH_LANDED_EMA,p2_R1_CLINCH_LANDED_EMA,p1_R1_CLINCH_ATTEMPTED_EMA,p2_R1_CLINCH_ATTEMPTED_EMA,p1_R1_GROUND_LANDED_EMA,p2_R1_GROUND_LANDED_EMA,p1_R1_GROUND_ATTEMPTED_EMA,p2_R1_GROUND_ATTEMPTED_EMA
8207,Scott Morris,Sean Daugherty,John McCarthy,1,1994-03-11,70.0,210.0,NaN,Orthodox,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72.0,175.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,18.26694,-2.0,NaN,35.0,NaN,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8195,Royce Gracie,Remco Pardoel,John McCarthy,1,1994-03-11,73.0,